# Data Analysis

## Preparing The Data

In [18]:
from pathlib import Path

import numpy as np
import pandas as pd

In [19]:
def data(root: Path) -> Path:
    return root / "data"


def output(root: Path) -> Path:
    return data(root) / "output"


def raw_results(root: Path) -> Path:
    return data(root) / "raw_results" / "raw_results.csv"


def subjects(root: Path) -> Path:
    return data(root) / "raw_results" / "subjects.csv"


def commits(root: Path) -> Path:
    return data(root) / "raw_results" / "commits.csv"


def truth(root: Path) -> Path:
    return data(root) / "truth" / "truth.csv"


In [20]:
raw_results_df = pd.read_csv(
    raw_results(Path.cwd()),
    low_memory=False,
)

In [21]:
df = raw_results_df.copy()

for column in [
    "full_name",
    "clone_url",
    "git_url",
    "teams_url",
    "sha",
    "url",
    "message",
]:
    if column in df.columns:
        df = df.drop(columns=[column])

In [22]:
if df["is_ccdc_event"].isna().any():
    n = int(df["is_ccdc_event"].isna().sum())
    print(f"WARNING: {n} rows have non-parseable is_ccdc_event -> NaN")

def to_bool(series: pd.Series) -> pd.Series:
    if series.dtype == bool:
        return series
    s = series.astype("string").str.strip().str.lower()
    mapping = {"true": True, "false": False}
    return s.map(mapping)

df["is_ccdc_event"] = to_bool(df["is_ccdc_event"])

In [23]:
df["detected_channel"] = df["detected_channel"].astype("string").fillna("")

In [24]:
key_cols = ["full_name_of_repo", "commit_sha", "path"]

gb = df.groupby(key_cols, dropna=False)

def agg_channels(x: pd.Series) -> tuple[str, ...]:
    vals = [
        v
        for v in x.astype("string").tolist()
        if isinstance(v, str) and v.strip() != ""
    ]
    return tuple(sorted(set(vals)))

agg = gb.agg(
    is_ccdc_event=("is_ccdc_event", "first"),
    date=("date", "first"),
    detected_channels=("detected_channel", agg_channels),
)

for column in [
    "id",
    "homepage",
    "created_at",
    "pushed_at",
    "updated_at",
    "has_discussions",
    "has_issues",
    "has_pages",
    "has_projects",
    "has_wiki",
    "forks_count",
    "open_issues_count",
    "stargazers_count",
    "subscribers_count",
    "size",
]:
    if column in df.columns:
        agg[column] = gb[column].first()

agg = agg.reset_index()

df = agg

In [25]:
date_cols = ["created_at", "pushed_at", "updated_at", "date"]

for column in date_cols:
    if column in df.columns:
        df[column] = pd.to_datetime(df[column], errors="coerce", utc=True)

In [26]:
rowwise_min = df[["date", "created_at"]].min(axis=1)

df["birthday"] = (
    rowwise_min
        .groupby(df["full_name_of_repo"], dropna=False)
        .transform("min")
)

In [27]:
# key_cols = ["full_name_of_repo"]

# df["birthday"] = (
#     df.groupby(key_cols, dropna=False)["date"]
#       .transform("min")
# )

# expected = (
#     df.groupby("full_name_of_repo", dropna=False)["date"]
#       .min()
# )

# actual = (
#     df.groupby("full_name_of_repo", dropna=False)["birthday"]
#       .first()
# )

# assert expected.equals(actual), "ERROR"

In [28]:
df["age_in_days"] = (
    (df["date"] - df["birthday"]).to_numpy()
    / np.timedelta64(1, "D")
)

In [29]:
AGE_GROUPS_YEARS = [
    (0, 1, "0-1"),
    (1, 2, "1-2"),
    (2, 3, "2-3"),
    (3, 4, "3-4"),
    (4, 5, "4-5"),
    (5, 6, "5-6"),
    (6, 7, "6-7"),
    (7, 8, "7-8"),
    (8, 9, "8-9"),
    (9, 10, "9-10"),
    (10, 11, "10-11"),
    (11, 12, "11-12"),
    (12, 13, "12-13"),
    (13, 14, "13-14"),
    (14, 15, "14-15"),
    (15, 999, "14+"),
]

def assign_age_group(age_in_days: float) -> str:
    if pd.isna(age_in_days):
        return "unknown"
    for lo, hi, label in AGE_GROUPS_YEARS:
        if age_in_days >= lo * 365.25 and age_in_days < hi * 365.25:
            return label
    return "unknown"

df["repo_age_group_at_commit"] = df["age_in_days"].map(assign_age_group)

df["repo_age_group_at_commit"].value_counts(dropna=False)

repo_age_group_at_commit
0-1      6397
1-2      2430
2-3      1995
3-4      1439
4-5      1214
5-6      1024
6-7       895
7-8       674
8-9       586
9-10      377
10-11     320
11-12     163
12-13     120
13-14      42
14-15      11
14+         7
Name: count, dtype: int64

In [30]:
subjects = df.copy()

for column in [
    "id",
    "homepage",
    "pushed_at",
    "updated_at",
    "has_discussions",
    "has_issues",
    "has_pages",
    "has_projects",
    "has_wiki",
    "forks_count",
    "open_issues_count",
    "stargazers_count",
    "subscribers_count",
    "size",
    "birthday",
]:
    if column in subjects.columns:
        subjects = subjects.drop(columns=[column])

In [31]:
subjects = subjects.sort_values(
    by=["full_name_of_repo", "date", "commit_sha", "path"],
    ascending=[True, True, True, True],
).reset_index(drop=True)

subjects.head(30)

,full_name_of_repo,commit_sha,path,is_ccdc_event,date,detected_channels,created_at,age_in_days,repo_age_group_at_commit
0,05bit/peewee-async,fc8edc1d35cbda8477ceb4dd672d2ed115b18635,README.md,False,2014-09-27 22:15:28+00:00,(),2014-09-27 22:15:28+00:00,0.000000,0-1
1,05bit/peewee-async,03d7da284e47d3018e071995e2d9cd7a8625f41e,README.md,True,2014-09-28 13:44:56+00:00,"(issues,)",2014-09-27 22:15:28+00:00,0.645463,0-1
2,05bit/peewee-async,53130218e08eaacf34f3d68338bb008fdfeb6c72,README.md,False,2014-09-28 13:49:37+00:00,(),2014-09-27 22:15:28+00:00,0.648715,0-1
3,05bit/peewee-async,c90ae2ca4a4d6cdac078e9918c53e9ea0ddcb632,README.md,False,2014-09-29 10:33:36+00:00,(),2014-09-27 22:15:28+00:00,1.512593,0-1
4,05bit/peewee-async,4db3f204d4ee60a91146e27d3d82b9edfc1086c2,README.md,False,2014-10-11 09:08:29+00:00,(),2014-09-27 22:15:28+00:00,13.453484,0-1
5,05bit/peewee-async,f546a0d2c41b2c29f718ec4f6459c44eedd00983,README.md,False,2014-10-11 12:07:44+00:00,(),2014-09-27 22:15:28+00:00,13.577963,0-1
6,05bit/peewee-async,95506301a3c0279812ee6802e30f1741435c5aca,README.md,False,2014-10-11 12:11:48+00:00,(),2014-09-27 22:15:28+00:00,13.580787,0-1
7,05bit/peewee-async,eead5b75cf1ef4c06cfce3d09184230b4e64ffa8,README.md,False,2014-10-11 14:38:03+00:00,(),2014-09-27 22:15:28+00:00,13.682350,0-1
8,05bit/peewee-async,7bce99a67f17685aeec8f756908f8ad19a2cd82e,README.md,False,2014-10-11 15:09:02+00:00,(),2014-09-27 22:15:28+00:00,13.703866,0-1
9,05bit/peewee-async,0b25d439c214c32d531928a5114593a6db7762b8,README.md,False,2014-10-11 15:39:33+00:00,(),2014-09-27 22:15:28+00:00,13.725058,0-1


In [32]:
repos = df.groupby("full_name_of_repo").agg(
    id=("id", "first"),
    n_subjects=("commit_sha", "count"),
    n_ccdc_events=("is_ccdc_event", lambda s: int(s.fillna(False).sum())),
    ccdc_rate=("is_ccdc_event", lambda s: float(s.fillna(False).mean())),
    birthday=("birthday", "first"),
    pushed_at=("pushed_at", "first"),
    updated_at=("updated_at", "first"),
    homepage=("homepage", "first"),
    has_discussions=("has_discussions", "first"),
    has_issues=("has_issues", "first"),
    has_pages=("has_pages", "first"),
    has_projects=("has_projects", "first"),
    has_wiki=("has_wiki", "first"),
    forks_count=("forks_count", "first"),
    open_issues_count=("open_issues_count", "first"),
    stargazers_count=("stargazers_count", "first"),
    subscribers_count=("subscribers_count", "first"),
    size=("size", "first"),
).reset_index()

repos["last_activity"] = repos[["pushed_at", "updated_at"]].max(axis=1)

repos["age"] = repos["last_activity"] - repos["birthday"]

expected = len(subjects)
actual = repos["n_subjects"].sum()
assert expected == actual, "ERROR"

## Descriptive Statistics

### Overall, how many of this and that?

In [33]:
print(f"How many repos? {len(repos)}")

active_repos = repos[
    repos["last_activity"].dt.year == 2025
].copy()
n_active_repos = len(active_repos)
print(f"How many repos are still active? {n_active_repos}")

n_commits = subjects["commit_sha"].nunique()
print(f"How many commits? {n_commits}")

n_paths = subjects["path"].nunique()
print(f"How many distinct paths? {n_paths}")

n_subjects = len(subjects)
print(f"How many subjects? {n_subjects}")

s = subjects["is_ccdc_event"]
n_positives = s.sum()
print(f"How many positives? {n_positives}")
n_negatives = (~s).sum()
print(f"How many negatives? {n_negatives}")
print(f"What is the rate of positives? {n_positives / n_subjects}")
assert n_subjects == n_positives + n_negatives, "ERROR"

unique_channels = (
    subjects["detected_channels"]
    .explode()
    .dropna()
    .unique()
)
n_unique_channels = len(unique_channels)
print(f"How many distinct channels? {n_unique_channels}")

n_positives_with_channel = (
    (df["is_ccdc_event"] == True)
    & (df["detected_channels"] != ())
).sum()
print(f"How many positives with a channel? {n_positives_with_channel}")
n_positives_without_channel = (
    (df["is_ccdc_event"] == True)
    & (df["detected_channels"] == ())
).sum()
print(f"How many positives without a channel? {n_positives_without_channel}")

n_negatives_with_channel = (
    (df["is_ccdc_event"] == False)
    & (df["detected_channels"] != ())
).sum()
print(f"How many negatives with a channel? {n_negatives_with_channel}")
n_negatives_without_channel = (
    (df["is_ccdc_event"] == False)
    & (df["detected_channels"] == ())
).sum()
print(f"How many negatives without a channel? {n_negatives_without_channel}")
assert n_subjects == n_positives_with_channel + n_positives_without_channel + n_negatives_with_channel + n_negatives_without_channel, "ERROR"



How many repos? 364
How many repos are still active? 143
How many commits? 17523
How many distinct paths? 5
How many subjects? 17694
How many positives? 3626
How many negatives? 14068
What is the rate of positives? 0.20492822425681023
How many distinct channels? 33
How many positives with a channel? 1691
How many positives without a channel? 1935
How many negatives with a channel? 1909
How many negatives without a channel? 12159


### Outliers – overall

In [34]:
print(f"The repo with the earliest birthday? {repos["birthday"].min()}")
print(f"The repo with the latest birthday? {repos["birthday"].max()}")

print(f"The oldest repo? {repos.loc[repos["age"].idxmax(), "full_name_of_repo"]}")

earliest_commit = subjects["date"].idxmin()
earliest_commit_repo = subjects.loc[earliest_commit, "full_name_of_repo"]
earliest_commit_sha = subjects.loc[earliest_commit, "commit_sha"]
print(f"Earliest commit? {earliest_commit_sha}@{earliest_commit_repo}")
latest_commit = subjects["age_in_days"].idxmax()
latest_commit_repo = subjects.loc[latest_commit, "full_name_of_repo"]
latest_commit_sha = subjects.loc[latest_commit, "commit_sha"]
print(f"Oldest commit? {latest_commit_sha}@{latest_commit_repo}")

path_counts = (
    subjects["path"]
    .value_counts()
)
print(f"Most popular path? {path_counts.idxmax()} with {path_counts.max()} counts")
print(f"Least popular path? {path_counts.idxmin()} with {path_counts.min()} counts")

channel_counts = (
    subjects["detected_channels"]
    .explode()
    .dropna()
    .value_counts()
)
print(f"Most frequently detected channel? {channel_counts.idxmax()} with {channel_counts.max()} counts")
print(f"Least commonly recognized channel? {channel_counts.idxmin()} with {channel_counts.min()} counts")

The repo with the earliest birthday? 2008-08-04 11:09:52+00:00
The repo with the latest birthday? 2022-11-12 18:36:31+00:00
The oldest repo? gctools-outilsgc/gcconnex
Earliest commit? 77533b76fbc2f0fd72445f8f3afb5d5278d4f4aa@gctools-outilsgc/gcconnex
Oldest commit? 8f00f28a3fe1629d6fae66507ad9af30f7698f58@twilio/twilio-java
Most popular path? README.md with 16172 counts
Least popular path? contributing.md with 27 counts
Most frequently detected channel? form with 643 counts
Least commonly recognized channel? zulip with 2 counts


### Across each and every repo, Over the years…

- For each year, since the first commit:
    - How many repos (were born in that year)?
    - How many commits?
    - How many distinct paths?
    - How many subjects?
    - How many positives?
    - How many negatives?
    - What is the rate of positives?
    - How many distinct channels?
    - How many subjects without a channel?
    - How many positives without a channel?
    - How many negatives with a channel?
    - Most popular path
    - Least popular path
    - Most often detected channel
    - Rarest channel
    - How many channels per positive? Boxplot, Histogram

### Repo specific boxplots

- Birthday
- Age – max(pushed_at, updated_at) - birthday
- How many commits?
- How many distinct paths?
- How many subjects?
- How many positives?
- How many negatives?
- What is the rate of positives?
- How many distinct channels?
- How many subjects without a channel?
- How many positives without a channel?
- How many negatives with a channel?
- How many channels per positive? Boxplot, Histogram

### Within a repo, Over the years…

- For each repo age group:
    - How many repos?
    - How many commits?
    - How many distinct paths?
    - How many subjects?
    - How many positives?
    - How many negatives?
    - What is the rate of positives?
    - How many distinct channels?
    - How many subjects without a channel?
    - How many positives without a channel?
    - How many negatives with a channel?
    - Most popular path
    - Least popular path
    - Most often detected channel
    - Rarest channel
    - How many channels per positive? Boxplot, Histogram

### Further Path analysis

- For each and every distinct path incl. suffix:
    - overall
        - How many repos?
        - How many subjects?
        - How many positives?
        - How many negatives?
        - What is the rate of positives?
        - How many distinct channels?
        - How many subjects without a channel?
        - How many positives without a channel?
        - How many negatives with a channel?
        - Most often detected channel
        - Rarest channel
        - How many channels per positive?
    - per repo
        - How many subjects? Boxplot
        - How many positives? Boxplot
        - How many negatives? Boxplot
        - What is the rate of positives? Boxplot
        - How many distinct channels? Boxplot
        - How many subjects without a channel? Boxplot
        - How many positives without a channel? Boxplot
        - How many negatives with a channel? Boxplot
        - How many channels per positive? Boxplot
    - per year
        - How many repos?
        - How many subjects?
        - How many positives?
        - How many negatives?
        - What is the rate of positives?
        - How many distinct channels?
        - How many subjects without a channel?
        - How many positives without a channel?
        - How many negatives with a channel?
        - How many channels per positive?
    - per repo age group
        - How many repos?
        - How many subjects?
        - How many positives?
        - How many negatives?
        - What is the rate of positives?
        - How many distinct channels?
        - How many subjects without a channel?
        - How many positives without a channel?
        - How many negatives with a channel?
        - How many channels per positive?

- for each path suffix (e.g., ".txt", ".md"):
    - per year:
        - How many subjects? – popularity of txt vs. md

### Further Channel analysis

- overall:
    - top N most often detected channels
    - top N rarest channels
- per year:
    - top N most often detected channels – race of the channels…
    - top N rarest channels
- per repo age group:
    - top N most often detected channels
    - top N rarest channels
- for each distinct channel:
    - first time
    - last time
    - How many repos?
    - How many subjects?
    - …

### Correlations?

- What indicates a higher ccdc_rate?
    - The year?
    - The age of the repo?
    - The number of channels in use??
    - The ccdc_rate itself???
    - The number of channels detected for that subject?